In [133]:
import numpy as np

In [134]:
def find_inv(inv_matrix: list[list[float]], x: list[float], col: int):
    n = len(inv_matrix)
    l = [0 for i in range(n)]
    for i in range(n):
        s = 0
        for j in range(n):
            s += inv_matrix[i][j] * x[j]
        l[i] = s

    if l[col] == 0:
        raise Exception(f"Matrix is not inversable: l = {l}")

    l_col = l[col]
    l[col] = -1
    
    l = list(map(lambda x: -1 / l_col * x, l))

    q = []
    for i in range(n):
        q.append([0] * n)

    for i in range(n):
        q[i][i] = 1
        q[i][col] = l[i]

    ans = []
    for i in range(n):
        ans.append([0] * n)

    for i in range(n):
        for j in range(n):
            t1 = q[i][i] * inv_matrix[i][j]
            t2 = q[i][col] * inv_matrix[col][j]
            if i == col:
                t2 = 0
            ans[i][j] = t1 + t2

    return ans

In [135]:
def simplex_method_phase_2(A: list[list[float]], c: list[float], x0: list[float], base_indicies: list[float]):
    
    A_b = [[row[j] for j in base_indicies] for row in A]
    A_b = np.array(A_b)
    
    A_b_inv = np.linalg.inv(A_b)
    c_b = np.array([c[i] for i in base_indicies])
    
    num_of_base = len(base_indicies)
    
    while (True):
        u = c_b.dot(A_b_inv)
        delta = u.dot(np.array(A)) - np.array(c)
        j0 = -1
        for i in range(len(delta)):
            if delta[i] < 0:
                j0 = i
                break

        if j0 == -1:
            return (x0, base_indicies, A_b_inv)

        A_j0 = np.array([row[j0] for row in A])

        z = A_b_inv.dot(A_j0)

        teta_min = np.inf
        teta_min_ind = -1
        for i in range(num_of_base):
            teta = x0[base_indicies[i]] / z[i] if z[i] > 0 else np.inf
            if teta < teta_min:
                teta_min = teta
                teta_min_ind = i

        if teta_min_ind == -1:
            raise Exception("The objective function is not limited on the set of acceptable plans")
        
        new_base_ind = j0
        old_base_ind = base_indicies[teta_min_ind]
        base_indicies[teta_min_ind] = new_base_ind

        x0[new_base_ind] = teta_min
        x0[old_base_ind] = 0
        
        for i in range(num_of_base):
            if base_indicies[i] != new_base_ind:
                x0[base_indicies[i]] -= teta_min * z[i]
                
        new_column = [row[new_base_ind] for row in A]

        A_b_inv = np.array(find_inv(A_b_inv, new_column, teta_min_ind))
        c_b[teta_min_ind] = c[new_base_ind]
    
    

In [136]:
def simplex_method_phase_1(A: list[list[float]], b: list[float], c: list[float]):
    n = len(A[0])
    m = len(A)
    
    for i in range(m):
        if b[i] < 0:
            A[i] = list(map(lambda x: -x, A[i]))
    
    help_A = A.copy()
    E = [[1 if i == j else 0 for i in range(m)] for j in range(m)]
    for i in range(m):
        help_A[i].extend(E[i])
    
    help_c = [0 if i < n else -1 for i in range(n + m)]
    help_x = [0 for i in range(n)] + b
    help_base = [n + i for i in range(m)]
    
    (help_solution, help_base, help_A_b_inv) = simplex_method_phase_2(help_A.copy(), help_c.copy(), help_x.copy(), help_base.copy())
    
    if any(help_solution[n:]):
        raise Exception("Task is incompatible")
        
    base = help_base
    
    while not max(base) < n:
        j_k, k = max([(val, ind) for ind, val in enumerate(base)])
        i = j_k - n
        
        found = False
        for j in set([i for i in range(n)]).difference(set(base)):
            l = [0 for i in range(m)]
            help_A_j = [row[j] for row in help_A]
            for ii in range(m):
                s = 0
                for jj in range(m):
                    s += help_A_b_inv[ii][jj] * help_A_j[jj]
                l[i] = s
            if l[k] != 0:
                base[k] = j
                help_A_b_inv = find_inv(help_A_b_inv, help_A_j, k)
                found = True
        
        if not found:
            base.remove(j_k)
            b = b[:i] + b[i + 1:]
            A = [row for ind, row in enumerate(A) if ind != i]
            help_A = [row[:j_k] + row[j_k + 1:] for ind, row in enumerate(help_A) if ind != i]
            help_A_b = [[row[ind] for ind in base] for row in help_A]
            help_A_b_inv = np.linalg.inv(np.array(help_A_b))
    
    return help_solution[:n], base
    

In [139]:
A = [[-1, 1, 1, 0, 0],
     [ 1, 0, 0, 1, 0],
     [ 0, 1, 0, 0, 1]]
b = [1, 3, 2]
c = [1, 1, 0, 0, 0]

res = simplex_method_phase_1(A, b, c)
print(res)


A = [[1, 1, 1],
     [2, 2, 2]]
b = [0, 0]
c = [1, 0, 0]

res = simplex_method_phase_1(A, b, c)
print(res)


A = [[1, 1, 1],
     [2, 2, 2]]
b = [-1, 0]
c = [1, 0, 0]

res = simplex_method_phase_1(A, b, c)
print(res)

([3.0, 2.0, 2.0, 0, 0], [1, 2, 0])
([0.0, 0, 0], [0])


Exception: Task is incompatible